<a href="https://colab.research.google.com/github/MohamedHamed12/web_scraping/blob/main/scrap_netmeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google_colab_selenium

In [ ]:
import time
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from urllib.parse import urlparse
from urllib.request import urlretrieve

In [ ]:
import google_colab_selenium as gs

driver = gs.Chrome()



In [ ]:
url='https://www.netmeds.com/'
response = requests.get(url)
if response.status_code != 200:
    print("Failed to fetch data from the URL.")
    exit()
soup = BeautifulSoup(response.text, 'html.parser')
nav_sections = soup.find('div', class_='nav-sections')
categories_urls = [category.find('a')['href'] for category in nav_sections.find_all('li') if category.find('a')['href']]

# class="nav-sections"

In [ ]:
def save_files(image_url, folder_name, category_name):
        # if not os.path.exists(folder_name
        print(image_url,category_name)
        if not os.path.exists(folder_name):
          os.makedirs(folder_name, exist_ok=True)


        filename = f"{category_name}.jpg"
        image_path = os.path.join(folder_name, filename)
        urlretrieve(image_url, image_path)


        data = {
            'Category_name': category_name,
            'Image': filename

        }
        # df = pd.DataFrame([data], index=[0])  # Add an index of 0

        df = pd.DataFrame([data])
        csv_file = 'catagory_data.csv'

        if os.path.exists(csv_file):
            df.to_csv(csv_file, mode='a', index=False, header=False)
        else:
            df.to_csv(csv_file, index=False, header=True)

        print("Data extraction completed.")

In [ ]:
def get_category_data(url):
    # Send a GET request to the URL
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from {url}")
        return None

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    category_name = url.split('/')[-1]
    # Find the image element
    image_body=soup.find('div',class_='category-main-banner')
    if not image_body:
        return category_name
    image_url = [image['src'] for image in image_body.find_all('img')][0]
    # if not image_url:
    #     return category_name
    save_files(image_url,'catagory',category_name)
    return category_name
for url in categories_urls:
    get_category_data('https://www.netmeds.com'+url)

https://www.netmeds.com/images/cms/aw_rbslider/slides/1715853034_Neurobion_1000x200.png covid-essentials
Data extraction completed.
https://www.netmeds.com/images/cms/aw_rbslider/slides/1707916702_Category_Banner_Web_Portfolio.png diabetes-support
Data extraction completed.
https://www.netmeds.com/images/category/v1/916/cardiac_care_5.jpg cardiac-care
Data extraction completed.
https://www.netmeds.com/images/cms/aw_rbslider/slides/1717675731_Sexual_Health.jpg ayurvedic
Data extraction completed.
https://www.netmeds.com/images/cms/aw_rbslider/slides/1716968586_1000x200_(1).jpg homeopathy
Data extraction completed.
https://www.netmeds.com/images/cms/aw_rbslider/slides/1695200613_Surgical.jpg surgical
Data extraction completed.
https://www.netmeds.com/images/cms/aw_rbslider/slides/1717676089_Skin_Care.jpg skin-care
Data extraction completed.


In [ ]:
# !rm -rf catagory
# !rm -rf catagory_data.csv

In [ ]:

def get_product_urls(category_url):
    driver = gs.Chrome()
    url = category_url

    print(url)
    # Open the webpage
    driver.get(url)

    # Simulate scrolling to load more content
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(2)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parse the HTML content
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Find all product containers
    scope = soup.find("div", class_="all-product")
    if not scope:
        print("No product containers found.")
        return []
    product_containers = scope.find_all("div", class_="cat-item")

    # Iterate over each product container
    urls= []
    for product in product_containers:
        url = product.find("a", class_="category_name")["href"]
        urls.append(url)
    driver.quit()
    return urls

catagory_products={}
for category_url in categories_urls:
  urls=get_product_urls('https://www.netmeds.com'+category_url)
  catagory_products[category_url.split('/')[-1]]=urls




In [ ]:


def extract_data(url,catagory_name):
    # Send a GET request to the URL
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extracting product name
        product_name = soup.find('h1', class_='black-txt').text.strip()
        # Extracting price
        price = soup.find('span', class_='final-price').text.strip()


        # Extracting image URLs
        images_body=soup.find('div',class_='product-media')

        image_urls = [image['src'] for image in images_body.find_all('img')]


        # Create a folder to store images
        folder_name = 'product_images'
        os.makedirs(folder_name, exist_ok=True)

        # Download images
        files_names = []

        for i, image_url in enumerate(image_urls[:2]):
            filename = f"{product_name.replace(' ', '_')}_{i}.jpg"
            image_path = os.path.join(folder_name, filename)
            try:
              urlretrieve(image_url, image_path)
            except Exception as e:
              continue
            files_names.append(filename)
            # print(f"Image {i+1} downloaded: {image_path}")

        # Save data to CSV with header
        data = {
            'Category': catagory_name,
            'Product Name': product_name,
                'Price': price,

                'Images': [', '.join(files_names)]}

        # df = pd.DataFrame(data)
        # df.to_csv('product_data.csv', index=False, header=True)
        # Append data to the CSV file
        # new_data = {'Product Name': product_name, 'Price': price}  # Add more data here
        df = pd.DataFrame([data])
        if os.path.exists('product_data.csv'):
            df.to_csv('product_data.csv', mode='a', index=False, header=False)
        else:
            df.to_csv('product_data.csv', index=False, header=True)

        print("Data extraction completed.")
    else:
        print(f"Failed to fetch data from {url}")

for catagory_name,products in catagory_products.items():
  for url in products[0:30]:
      extract_data('https://www.netmeds.com/'+url,catagory_name)


In [ ]:


def fetch_html(url):
    """Fetch HTML content from the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch HTML content from {url}")
        return None



In [ ]:
def extract_data(url, category_name):
    """Extract product data from the provided URL."""
    html_content = fetch_html(url)
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        product_name = soup.find('h1', class_='black-txt').text.strip()
        price = soup.find('span', class_='final-price').text.strip()
        images_body = soup.find('div', class_='product-media')
        image_urls = [image['src'] for image in images_body.find_all('img')]
        folder_name = 'product_images'
        os.makedirs(folder_name, exist_ok=True)
        files_names = []

        for i, image_url in enumerate(image_urls[:2]):
            filename = f"{product_name.replace(' ', '_')}_{i}.jpg"
            image_path = os.path.join(folder_name, filename)
            try:
              urlretrieve(image_url, image_path)
            except Exception as e:
              continue
            files_names.append(filename)

        data = {
            'Category': [category_name],
            'Product Name': [product_name],
            'Price': [price],
            'Images': [', '.join(files_names)]
        }

        df = pd.DataFrame(data)
        csv_file = 'product_data.csv'

        if os.path.exists(csv_file):
            df.to_csv(csv_file, mode='a', index=False, header=False)
        else:
            df.to_csv(csv_file, index=False, header=True)

        # print("Data extraction completed.")
    else:
        print(f"Data extraction failed for URL: {url}")




In [ ]:
def scrape_categories(catagory_products):
    """Scrape product data for each category."""
    for category_name, products in catagory_products.items():
        print(f"Scraping {category_name}...")
        for url in products[:30]:
            full_url = 'https://www.netmeds.com/' + url
            extract_data(full_url, category_name)
        print(f"Scraping {category_name} completed.")




In [ ]:

scrape_categories(catagory_products)

In [ ]:
!du -sh product_images
!du -sh catagory
!du -sh product_data.csv
!du -sh catagory_data.csv

In [ ]:
!du -sh product_data.csv

In [ ]:
# !ls
!rm -rf product_images
!rm -rf product_data.csv

In [ ]:
!cp -r product_images drive/MyDrive/sc
!cp -r catagory drive/MyDrive/sc
!cp -r product_data.csv drive/MyDrive/sc
!cp -r catagory_data.csv drive/MyDrive/sc